In [51]:
import pandas as pd
import os

# Define paths to the detailed and summary data
base_detailed_dir = "/Users/oe/BPS_Project/Graphene_BP/predictions_ada/ADA_0_NoShuffleCV_hgcp"
summary_data_path = "/Users/oe/BPS_Project/Graphene_BP/predictions_ada/noshuffle_hgcp.csv"

# Function to process detailed data for a single subject
def process_subject_data(subject_id, bp_type='DBP'):
    file_path = os.path.join(base_detailed_dir, f"subject_id_{subject_id}", "NoShuffleCV_hgcp_NoIBI_ada_mean_test.csv")
    detailed_data = pd.read_csv(file_path)
    errors = detailed_data[f'Predict_data_{bp_type}'] - detailed_data[f'Ref_data_{bp_type}']
    sde = errors.std()
    bp_mean = detailed_data[f'Ref_data_{bp_type}'].mean()
    bp_range = detailed_data[f'Ref_data_{bp_type}'].max() - detailed_data[f'Ref_data_{bp_type}'].min()
    return sde, bp_mean, bp_range

# Load the summary data
summary_data = pd.read_csv(summary_data_path).set_index('Subject ID')

# Initialize tables for DBP, SBP, and MAP
dbp_table = pd.DataFrame(index=summary_data.index)
sbp_table = pd.DataFrame(index=summary_data.index)
map_table = pd.DataFrame(index=summary_data.index)

# Process detailed data and compute summary statistics
for subject_id in summary_data.index:
    dbp_sde, dbp_mean, dbp_range = process_subject_data(subject_id, 'DBP')
    sbp_sde, sbp_mean, sbp_range = process_subject_data(subject_id, 'SBP')
    
    # Populate DBP table
    dbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.1']
    dbp_table.loc[subject_id, 'Sde'] = dbp_sde
    dbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.1'] + 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.1'] - 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.1']
    dbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.1']
    dbp_table.loc[subject_id, 'BP Range'] = dbp_range
    dbp_table.loc[subject_id, 'BP mean'] = dbp_mean
    dbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing DBP N']
    
    # Populate SBP table
    sbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.3']
    sbp_table.loc[subject_id, 'Sde'] = sbp_sde
    sbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.3'] + 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.3'] - 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.3']
    sbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.3']
    sbp_table.loc[subject_id, 'BP Range'] = sbp_range
    sbp_table.loc[subject_id, 'BP mean'] = sbp_mean
    sbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing SBP N']


    
    # Calculate and populate MAP table by the standard formula (MAP = DBP + 1/3 (SBP - DBP)) from the DBP and SBP tables
    map_table.loc[subject_id, 'ME'] = dbp_table.loc[subject_id, 'ME'] + 1/3 * (sbp_table.loc[subject_id, 'ME'] - dbp_table.loc[subject_id, 'ME'])
    map_table.loc[subject_id, 'Sde'] = (dbp_table.loc[subject_id, 'Sde'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'Sde'] ** 2) ** 0.5
    map_table.loc[subject_id, '95% CI +'] = map_table.loc[subject_id, 'ME'] + 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, '95% CI -'] = map_table.loc[subject_id, 'ME'] - 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, 'r'] = (dbp_table.loc[subject_id, 'r'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'r'] ** 2) ** 0.5
    map_table.loc[subject_id, 'RMSE'] = (dbp_table.loc[subject_id, 'RMSE'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'RMSE'] ** 2) ** 0.5
    map_table.loc[subject_id, 'BP Range'] = dbp_table.loc[subject_id, 'BP Range'] + 1/3 * (sbp_table.loc[subject_id, 'BP Range'] - dbp_table.loc[subject_id, 'BP Range'])
    map_table.loc[subject_id, 'BP mean'] = dbp_table.loc[subject_id, 'BP mean'] + 1/3 * (sbp_table.loc[subject_id, 'BP mean'] - dbp_table.loc[subject_id, 'BP mean'])
    map_table.loc[subject_id, 'N'] = dbp_table.loc[subject_id, 'N']

# Compute averages and standard deviations for each table
for table in [dbp_table, sbp_table, map_table]:
    table.loc['Average'] = table.mean(numeric_only=True)
    table.loc['STD'] = table.std(numeric_only=True)



# Output the final tables with Subject ID as the index
print("Diastolic Blood Pressure (DBP) Table:")
print(dbp_table)
print("\nSystolic Blood Pressure (SBP) Table:")
print(sbp_table)
print("\nMean Arterial Pressure (MAP) Table:")
print(map_table)





Diastolic Blood Pressure (DBP) Table:
                  ME        Sde  95% CI +  95% CI -         r       RMSE  \
Subject ID                                                                 
1          -0.520537   5.807699 -0.262629 -0.778446  0.766291   5.829495   
2           2.114587  10.955350  2.485631  1.743544  0.171170  11.155956   
3           0.525611   5.402667  0.751631  0.299591  0.641548   5.426950   
4          -0.707163   6.722196 -0.466170 -0.948156  0.863394   6.758172   
5           1.544792  11.326954  2.007309  1.082275  0.621157  11.429373   
6          -0.760199   7.164953 -0.437853 -1.082544  0.643132   7.203292   
Average     0.366182   7.896637  0.679653  0.052711  0.617782   7.967206   
STD         1.131263   2.367172  1.188769  1.077128  0.217202   2.422903   

             BP Range    BP mean            N  
Subject ID                                     
1           45.552252  83.493821  1948.000000  
2           44.972805  94.446071  3349.000000  
3        

In [39]:
import pandas as pd
import os

# Define paths to the detailed and summary data
base_detailed_dir = "/Users/oe/BPS_Project/Graphene_BP/predictions_ada/ADA_0_ShuffleCV_hgcp"
summary_data_path = "/Users/oe/BPS_Project/Graphene_BP/predictions_ada/shuffle_hgcp.csv"

# Function to process detailed data for a single subject
def process_subject_data(subject_id, bp_type='DBP'):
    file_path = os.path.join(base_detailed_dir, f"subject_id_{subject_id}", "ShuffleCV_hgcp_NoIBI_ada_mean_test.csv")
    detailed_data = pd.read_csv(file_path)
    errors = detailed_data[f'Predict_data_{bp_type}'] - detailed_data[f'Ref_data_{bp_type}']
    sde = errors.std()
    bp_mean = detailed_data[f'Ref_data_{bp_type}'].mean()
    bp_range = detailed_data[f'Ref_data_{bp_type}'].max() - detailed_data[f'Ref_data_{bp_type}'].min()
    return sde, bp_mean, bp_range

# Load the summary data
summary_data = pd.read_csv(summary_data_path).set_index('Subject ID')

# Initialize tables for DBP, SBP, and MAP
dbp_table = pd.DataFrame(index=summary_data.index)
sbp_table = pd.DataFrame(index=summary_data.index)
map_table = pd.DataFrame(index=summary_data.index)

# Process detailed data and compute summary statistics
for subject_id in summary_data.index:
    dbp_sde, dbp_mean, dbp_range = process_subject_data(subject_id, 'DBP')
    sbp_sde, sbp_mean, sbp_range = process_subject_data(subject_id, 'SBP')
    
    # Populate DBP table
    dbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.1']
    dbp_table.loc[subject_id, 'Sde'] = dbp_sde
    dbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.1'] + 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.1'] - 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.1']
    dbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.1']
    dbp_table.loc[subject_id, 'BP Range'] = dbp_range
    dbp_table.loc[subject_id, 'BP mean'] = dbp_mean
    dbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing DBP N']
    
    # Populate SBP table
    sbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.3']
    sbp_table.loc[subject_id, 'Sde'] = sbp_sde
    sbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.3'] + 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.3'] - 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.3']
    sbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.3']
    sbp_table.loc[subject_id, 'BP Range'] = sbp_range
    sbp_table.loc[subject_id, 'BP mean'] = sbp_mean
    sbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing SBP N']


    
    # Calculate and populate MAP table by the standard formula (MAP = DBP + 1/3 (SBP - DBP)) from the DBP and SBP tables
    map_table.loc[subject_id, 'ME'] = dbp_table.loc[subject_id, 'ME'] + 1/3 * (sbp_table.loc[subject_id, 'ME'] - dbp_table.loc[subject_id, 'ME'])
    map_table.loc[subject_id, 'Sde'] = (dbp_table.loc[subject_id, 'Sde'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'Sde'] ** 2) ** 0.5
    map_table.loc[subject_id, '95% CI +'] = map_table.loc[subject_id, 'ME'] + 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, '95% CI -'] = map_table.loc[subject_id, 'ME'] - 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, 'r'] = (dbp_table.loc[subject_id, 'r'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'r'] ** 2) ** 0.5
    map_table.loc[subject_id, 'RMSE'] = (dbp_table.loc[subject_id, 'RMSE'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'RMSE'] ** 2) ** 0.5
    map_table.loc[subject_id, 'BP Range'] = dbp_table.loc[subject_id, 'BP Range'] + 1/3 * (sbp_table.loc[subject_id, 'BP Range'] - dbp_table.loc[subject_id, 'BP Range'])
    map_table.loc[subject_id, 'BP mean'] = dbp_table.loc[subject_id, 'BP mean'] + 1/3 * (sbp_table.loc[subject_id, 'BP mean'] - dbp_table.loc[subject_id, 'BP mean'])
    map_table.loc[subject_id, 'N'] = dbp_table.loc[subject_id, 'N']

# Compute averages and standard deviations for each table
for table in [dbp_table, sbp_table, map_table]:
    table.loc['Average'] = table.mean(numeric_only=True)
    table.loc['STD'] = table.std(numeric_only=True)



# Output the final tables with Subject ID as the index
print("Diastolic Blood Pressure (DBP) Table:")
print(dbp_table)
print("\nSystolic Blood Pressure (SBP) Table:")
print(sbp_table)
print("\nMean Arterial Pressure (MAP) Table:")
print(map_table)

Diastolic Blood Pressure (DBP) Table:
                  ME       Sde  95% CI +  95% CI -         r      RMSE  \
Subject ID                                                               
1          -0.370737  4.313276  0.230064 -0.971538  0.876342  4.318313   
2           0.053768  4.215610  0.507237 -0.399701  0.873739  4.209599   
3          -0.240047  4.017514  0.290841 -0.770934  0.799704  4.015555   
4           0.010870  4.451105  0.520542 -0.498801  0.942387  4.443516   
5          -0.043210  5.636629  0.685259 -0.771680  0.917972  5.624528   
6          -0.428203  5.190813  0.302247 -1.158653  0.827693  5.195095   
Average    -0.169593  4.637491  0.422698 -0.761884  0.872973  4.634434   
STD         0.187408  0.578310  0.160552  0.258827  0.048775  0.576557   

             BP Range    BP mean           N  
Subject ID                                    
1           44.892577  83.519360  198.000000  
2           42.829244  94.576622  332.000000  
3           31.502819  67.751020 

In [43]:
import pandas as pd
import os

# Define paths to the detailed and summary data
base_detailed_dir = "/Users/oe/BPS_Project/Graphene_BP/predictions_ada/ADA_06_Valsalva"
summary_data_path = "/Users/oe/BPS_Project/Graphene_BP/predictions_ada/valsalva.csv"

# Function to process detailed data for a single subject
def process_subject_data(subject_id, bp_type='DBP'):
    file_path = os.path.join(base_detailed_dir, f"subject_id_{subject_id}", "NoShuffleCV_valsalva_NoIBI_ada_mean_test.csv")
    detailed_data = pd.read_csv(file_path)
    errors = detailed_data[f'Predict_data_{bp_type}'] - detailed_data[f'Ref_data_{bp_type}']
    sde = errors.std()
    bp_mean = detailed_data[f'Ref_data_{bp_type}'].mean()
    bp_range = detailed_data[f'Ref_data_{bp_type}'].max() - detailed_data[f'Ref_data_{bp_type}'].min()
    return sde, bp_mean, bp_range

# Load the summary data
summary_data = pd.read_csv(summary_data_path).set_index('Subject ID')

# Initialize tables for DBP, SBP, and MAP
dbp_table = pd.DataFrame(index=summary_data.index)
sbp_table = pd.DataFrame(index=summary_data.index)
map_table = pd.DataFrame(index=summary_data.index)

# Process detailed data and compute summary statistics
for subject_id in summary_data.index:
    dbp_sde, dbp_mean, dbp_range = process_subject_data(subject_id, 'DBP')
    sbp_sde, sbp_mean, sbp_range = process_subject_data(subject_id, 'SBP')
    
    # Populate DBP table
    dbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.1']
    dbp_table.loc[subject_id, 'Sde'] = dbp_sde
    dbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.1'] + 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.1'] - 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.1']
    dbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.1']
    dbp_table.loc[subject_id, 'BP Range'] = dbp_range
    dbp_table.loc[subject_id, 'BP mean'] = dbp_mean
    dbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing DBP N']
    
    # Populate SBP table
    sbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.3']
    sbp_table.loc[subject_id, 'Sde'] = sbp_sde
    sbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.3'] + 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.3'] - 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.3']
    sbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.3']
    sbp_table.loc[subject_id, 'BP Range'] = sbp_range
    sbp_table.loc[subject_id, 'BP mean'] = sbp_mean
    sbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing SBP N']


    
    # Calculate and populate MAP table by the standard formula (MAP = DBP + 1/3 (SBP - DBP)) from the DBP and SBP tables
    map_table.loc[subject_id, 'ME'] = dbp_table.loc[subject_id, 'ME'] + 1/3 * (sbp_table.loc[subject_id, 'ME'] - dbp_table.loc[subject_id, 'ME'])
    map_table.loc[subject_id, 'Sde'] = (dbp_table.loc[subject_id, 'Sde'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'Sde'] ** 2) ** 0.5
    map_table.loc[subject_id, '95% CI +'] = map_table.loc[subject_id, 'ME'] + 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, '95% CI -'] = map_table.loc[subject_id, 'ME'] - 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, 'r'] = (dbp_table.loc[subject_id, 'r'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'r'] ** 2) ** 0.5
    map_table.loc[subject_id, 'RMSE'] = (dbp_table.loc[subject_id, 'RMSE'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'RMSE'] ** 2) ** 0.5
    map_table.loc[subject_id, 'BP Range'] = dbp_table.loc[subject_id, 'BP Range'] + 1/3 * (sbp_table.loc[subject_id, 'BP Range'] - dbp_table.loc[subject_id, 'BP Range'])
    map_table.loc[subject_id, 'BP mean'] = dbp_table.loc[subject_id, 'BP mean'] + 1/3 * (sbp_table.loc[subject_id, 'BP mean'] - dbp_table.loc[subject_id, 'BP mean'])
    map_table.loc[subject_id, 'N'] = dbp_table.loc[subject_id, 'N']

# Compute averages and standard deviations for each table
for table in [dbp_table, sbp_table, map_table]:
    table.loc['Average'] = table.mean(numeric_only=True)
    table.loc['STD'] = table.std(numeric_only=True)



# Output the final tables with Subject ID as the index
print("Diastolic Blood Pressure (DBP) Table:")
print(dbp_table)
print("\nSystolic Blood Pressure (SBP) Table:")
print(sbp_table)
print("\nMean Arterial Pressure (MAP) Table:")
print(map_table)

Diastolic Blood Pressure (DBP) Table:
                  ME       Sde  95% CI +  95% CI -         r      RMSE  \
Subject ID                                                               
1          -0.227875  4.602681  0.224889 -0.680639  0.805834  4.602525   
3          -0.387054  7.517947  0.525030 -1.299139  0.450540  7.513507   
4          -0.477956  7.681631  0.246426 -1.202338  0.689011  7.687607   
5           0.095323  8.553790  1.003222 -0.812576  0.749915  8.541770   
6          -0.652597  4.675889  0.502052 -1.807246  0.875896  4.684312   
Average    -0.330032  6.606388  0.500324 -1.160387  0.714239  6.605944   
STD         0.253197  1.644453  0.280659  0.397635  0.145581  1.639966   

             BP Range    BP mean           N  
Subject ID                                    
1           36.088912  77.359197  397.000000  
3           30.661532  67.357799  261.000000  
4           48.196444  76.650734  432.000000  
5           47.073075  84.114541  341.000000  
6           3

In [58]:
import pandas as pd
import os

# Define paths to the detailed and summary data
base_detailed_dir = "/Users/oe/BPS_Project/Graphene_BP/predictions_ada/ADA_04_NextDay"
summary_data_path = "/Users/oe/BPS_Project/Graphene_BP/predictions_ada/nextday.csv"

# Function to process detailed data for a single subject
def process_subject_data(subject_id, bp_type='DBP'):
    file_path = os.path.join(base_detailed_dir, f"subject_id_{subject_id}", "SingleTrain_S1All_TestNextDay_NoIBI_ada_mean_test.csv")
    detailed_data = pd.read_csv(file_path, header=0)
    errors = detailed_data[f'Predict_data_{bp_type}'] - detailed_data[f'Ref_data_{bp_type}']
    sde = errors.std()
    bp_mean = detailed_data[f'Ref_data_{bp_type}'].mean()
    bp_range = detailed_data[f'Ref_data_{bp_type}'].max() - detailed_data[f'Ref_data_{bp_type}'].min()
    return sde, bp_mean, bp_range

# Load the summary data
summary_data = pd.read_csv(summary_data_path).set_index('Subject ID')

# Initialize tables for DBP, SBP, and MAP
dbp_table = pd.DataFrame(index=summary_data.index)
sbp_table = pd.DataFrame(index=summary_data.index)
map_table = pd.DataFrame(index=summary_data.index)

# Process detailed data and compute summary statistics
for subject_id in summary_data.index:
    dbp_sde, dbp_mean, dbp_range = process_subject_data(subject_id, 'DBP')
    sbp_sde, sbp_mean, sbp_range = process_subject_data(subject_id, 'SBP')
    
    # Populate DBP table
    dbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.1']
    dbp_table.loc[subject_id, 'Sde'] = dbp_sde
    dbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.1'] + 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.1'] - 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.1']
    dbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.1']
    dbp_table.loc[subject_id, 'BP Range'] = dbp_range
    dbp_table.loc[subject_id, 'BP mean'] = dbp_mean
    dbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing DBP N']
    
    # Populate SBP table
    sbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.3']
    sbp_table.loc[subject_id, 'Sde'] = sbp_sde
    sbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.3'] + 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.3'] - 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.3']
    sbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.3']
    sbp_table.loc[subject_id, 'BP Range'] = sbp_range
    sbp_table.loc[subject_id, 'BP mean'] = sbp_mean
    sbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing SBP N']


    
    # Calculate and populate MAP table by the standard formula (MAP = DBP + 1/3 (SBP - DBP)) from the DBP and SBP tables
    map_table.loc[subject_id, 'ME'] = dbp_table.loc[subject_id, 'ME'] + 1/3 * (sbp_table.loc[subject_id, 'ME'] - dbp_table.loc[subject_id, 'ME'])
    map_table.loc[subject_id, 'Sde'] = (dbp_table.loc[subject_id, 'Sde'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'Sde'] ** 2) ** 0.5
    map_table.loc[subject_id, '95% CI +'] = map_table.loc[subject_id, 'ME'] + 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, '95% CI -'] = map_table.loc[subject_id, 'ME'] - 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, 'r'] = (dbp_table.loc[subject_id, 'r'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'r'] ** 2) ** 0.5
    map_table.loc[subject_id, 'RMSE'] = (dbp_table.loc[subject_id, 'RMSE'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'RMSE'] ** 2) ** 0.5
    map_table.loc[subject_id, 'BP Range'] = dbp_table.loc[subject_id, 'BP Range'] + 1/3 * (sbp_table.loc[subject_id, 'BP Range'] - dbp_table.loc[subject_id, 'BP Range'])
    map_table.loc[subject_id, 'BP mean'] = dbp_table.loc[subject_id, 'BP mean'] + 1/3 * (sbp_table.loc[subject_id, 'BP mean'] - dbp_table.loc[subject_id, 'BP mean'])
    map_table.loc[subject_id, 'N'] = dbp_table.loc[subject_id, 'N']

# Compute averages and standard deviations for each table
for table in [dbp_table, sbp_table, map_table]:
    table.loc['Average'] = table.mean(numeric_only=True)
    table.loc['STD'] = table.std(numeric_only=True)



# Output the final tables with Subject ID as the index
print("Diastolic Blood Pressure (DBP) Table:")
print(dbp_table)
print("\nSystolic Blood Pressure (SBP) Table:")
print(sbp_table)
print("\nMean Arterial Pressure (MAP) Table:")
print(map_table)

Diastolic Blood Pressure (DBP) Table:
                  ME      Sde  95% CI +  95% CI -         r     RMSE  \
Subject ID                                                             
1           1.692966  9.59456  2.550416  0.835517  0.290433  9.73295   
Average     1.692966  9.59456  2.550416  0.835517  0.290433  9.73295   
STD         0.000000  0.00000  0.000000  0.000000  0.000000  0.00000   

             BP Range    BP mean      N  
Subject ID                               
1           37.082103  80.440294  481.0  
Average     37.082103  80.440294  481.0  
STD          0.000000   0.000000    0.0  

Systolic Blood Pressure (SBP) Table:
                  ME        Sde  95% CI +  95% CI -         r       RMSE  \
Subject ID                                                                 
1          -5.433317  13.027009 -4.269115 -6.597519 -0.244482  14.102166   
Average    -5.433317  13.027009 -4.269115 -6.597519 -0.244482  14.102166   
STD         0.000000   0.000000  0.000000  0.0000

In [68]:
import pandas as pd
import os

# Define paths to the detailed and summary data
base_detailed_dir = "/Users/oe/BPS_Project/Graphene_BP/predictions_svr/SVM_Shuffle_hgcp"
summary_data_path = "/Users/oe/BPS_Project/Graphene_BP/predictions_svr/shuffle_hgcp.csv"

# Function to process detailed data for a single subject
def process_subject_data(subject_id, bp_type='DBP'):
    file_path = os.path.join(base_detailed_dir, f"subject_id_{subject_id}", "ShuffleCV_hgcp_NoIBI_svr_mean_test.csv")
    detailed_data = pd.read_csv(file_path)
    errors = detailed_data[f'Predict_data_{bp_type}'] - detailed_data[f'Ref_data_{bp_type}']
    sde = errors.std()
    bp_mean = detailed_data[f'Ref_data_{bp_type}'].mean()
    bp_range = detailed_data[f'Ref_data_{bp_type}'].max() - detailed_data[f'Ref_data_{bp_type}'].min()
    return sde, bp_mean, bp_range

# Load the summary data
summary_data = pd.read_csv(summary_data_path).set_index('Subject ID')

# Initialize tables for DBP, SBP, and MAP
dbp_table = pd.DataFrame(index=summary_data.index)
sbp_table = pd.DataFrame(index=summary_data.index)
map_table = pd.DataFrame(index=summary_data.index)

# Process detailed data and compute summary statistics
for subject_id in summary_data.index:
    dbp_sde, dbp_mean, dbp_range = process_subject_data(subject_id, 'DBP')
    sbp_sde, sbp_mean, sbp_range = process_subject_data(subject_id, 'SBP')
    
    # Populate DBP table
    dbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.1']
    dbp_table.loc[subject_id, 'Sde'] = dbp_sde
    dbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.1'] + 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.1'] - 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.1']
    dbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.1']
    dbp_table.loc[subject_id, 'BP Range'] = dbp_range
    dbp_table.loc[subject_id, 'BP mean'] = dbp_mean
    dbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing DBP N']
    
    # Populate SBP table
    sbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.3']
    sbp_table.loc[subject_id, 'Sde'] = sbp_sde
    sbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.3'] + 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.3'] - 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.3']
    sbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.3']
    sbp_table.loc[subject_id, 'BP Range'] = sbp_range
    sbp_table.loc[subject_id, 'BP mean'] = sbp_mean
    sbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing SBP N']


    
    # Calculate and populate MAP table by the standard formula (MAP = DBP + 1/3 (SBP - DBP)) from the DBP and SBP tables
    map_table.loc[subject_id, 'ME'] = dbp_table.loc[subject_id, 'ME'] + 1/3 * (sbp_table.loc[subject_id, 'ME'] - dbp_table.loc[subject_id, 'ME'])
    map_table.loc[subject_id, 'Sde'] = (dbp_table.loc[subject_id, 'Sde'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'Sde'] ** 2) ** 0.5
    map_table.loc[subject_id, '95% CI +'] = map_table.loc[subject_id, 'ME'] + 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, '95% CI -'] = map_table.loc[subject_id, 'ME'] - 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, 'r'] = (dbp_table.loc[subject_id, 'r'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'r'] ** 2) ** 0.5
    map_table.loc[subject_id, 'RMSE'] = (dbp_table.loc[subject_id, 'RMSE'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'RMSE'] ** 2) ** 0.5
    map_table.loc[subject_id, 'BP Range'] = dbp_table.loc[subject_id, 'BP Range'] + 1/3 * (sbp_table.loc[subject_id, 'BP Range'] - dbp_table.loc[subject_id, 'BP Range'])
    map_table.loc[subject_id, 'BP mean'] = dbp_table.loc[subject_id, 'BP mean'] + 1/3 * (sbp_table.loc[subject_id, 'BP mean'] - dbp_table.loc[subject_id, 'BP mean'])
    map_table.loc[subject_id, 'N'] = dbp_table.loc[subject_id, 'N']

# Compute averages and standard deviations for each table
for table in [dbp_table, sbp_table, map_table]:
    table.loc['Average'] = table.mean(numeric_only=True)
    table.loc['STD'] = table.std(numeric_only=True)



# Output the final tables with Subject ID as the index
print("Diastolic Blood Pressure (DBP) Table:")
print(dbp_table)
print("\nSystolic Blood Pressure (SBP) Table:")
print(sbp_table)
print("\nMean Arterial Pressure (MAP) Table:")
print(map_table)

Diastolic Blood Pressure (DBP) Table:
                  ME        Sde  95% CI +  95% CI -         r       RMSE  \
Subject ID                                                                 
1           1.008641   8.818117  2.236926 -0.219644  0.171282   8.853463   
2          -0.525812   8.641519  0.403747 -1.455372 -0.140447   8.644502   
3          -0.915335   6.731131 -0.025862 -1.804808  0.099596   6.777906   
4           0.154754  11.151729  1.431677 -1.122168  0.554393  11.133758   
5          -0.041113  13.367064  1.686427 -1.768654  0.392055  13.338037   
6          -1.243706   7.544478 -0.182048 -2.305363  0.608725   7.627093   
Average    -0.260429   9.375673  0.925144 -1.446001  0.280934   9.395793   
STD         0.741303   2.246564  0.909124  0.655776  0.263642   2.213670   

             BP Range    BP mean           N  
Subject ID                                    
1           44.892577  83.519360  198.000000  
2           42.829244  94.576622  332.000000  
3           3

In [69]:
import pandas as pd
import os

# Define paths to the detailed and summary data
base_detailed_dir = "/Users/oe/BPS_Project/Graphene_BP/predictions_svr/SVM_0_NoShuffle_hgcp"
summary_data_path = "/Users/oe/BPS_Project/Graphene_BP/predictions_svr/noshuffe_hgcp.csv"

# Function to process detailed data for a single subject
def process_subject_data(subject_id, bp_type='DBP'):
    file_path = os.path.join(base_detailed_dir, f"subject_id_{subject_id}", "NoShuffleCV_hgcp_NoIBI_svr_mean_test.csv")
    detailed_data = pd.read_csv(file_path)
    errors = detailed_data[f'Predict_data_{bp_type}'] - detailed_data[f'Ref_data_{bp_type}']
    sde = errors.std()
    bp_mean = detailed_data[f'Ref_data_{bp_type}'].mean()
    bp_range = detailed_data[f'Ref_data_{bp_type}'].max() - detailed_data[f'Ref_data_{bp_type}'].min()
    return sde, bp_mean, bp_range

# Load the summary data
summary_data = pd.read_csv(summary_data_path).set_index('Subject ID')

# Initialize tables for DBP, SBP, and MAP
dbp_table = pd.DataFrame(index=summary_data.index)
sbp_table = pd.DataFrame(index=summary_data.index)
map_table = pd.DataFrame(index=summary_data.index)

# Process detailed data and compute summary statistics
for subject_id in summary_data.index:
    dbp_sde, dbp_mean, dbp_range = process_subject_data(subject_id, 'DBP')
    sbp_sde, sbp_mean, sbp_range = process_subject_data(subject_id, 'SBP')
    
    # Populate DBP table
    dbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.1']
    dbp_table.loc[subject_id, 'Sde'] = dbp_sde
    dbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.1'] + 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.1'] - 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.1']
    dbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.1']
    dbp_table.loc[subject_id, 'BP Range'] = dbp_range
    dbp_table.loc[subject_id, 'BP mean'] = dbp_mean
    dbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing DBP N']
    
    # Populate SBP table
    sbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.3']
    sbp_table.loc[subject_id, 'Sde'] = sbp_sde
    sbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.3'] + 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.3'] - 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.3']
    sbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.3']
    sbp_table.loc[subject_id, 'BP Range'] = sbp_range
    sbp_table.loc[subject_id, 'BP mean'] = sbp_mean
    sbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing SBP N']


    
    # Calculate and populate MAP table by the standard formula (MAP = DBP + 1/3 (SBP - DBP)) from the DBP and SBP tables
    map_table.loc[subject_id, 'ME'] = dbp_table.loc[subject_id, 'ME'] + 1/3 * (sbp_table.loc[subject_id, 'ME'] - dbp_table.loc[subject_id, 'ME'])
    map_table.loc[subject_id, 'Sde'] = (dbp_table.loc[subject_id, 'Sde'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'Sde'] ** 2) ** 0.5
    map_table.loc[subject_id, '95% CI +'] = map_table.loc[subject_id, 'ME'] + 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, '95% CI -'] = map_table.loc[subject_id, 'ME'] - 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, 'r'] = (dbp_table.loc[subject_id, 'r'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'r'] ** 2) ** 0.5
    map_table.loc[subject_id, 'RMSE'] = (dbp_table.loc[subject_id, 'RMSE'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'RMSE'] ** 2) ** 0.5
    map_table.loc[subject_id, 'BP Range'] = dbp_table.loc[subject_id, 'BP Range'] + 1/3 * (sbp_table.loc[subject_id, 'BP Range'] - dbp_table.loc[subject_id, 'BP Range'])
    map_table.loc[subject_id, 'BP mean'] = dbp_table.loc[subject_id, 'BP mean'] + 1/3 * (sbp_table.loc[subject_id, 'BP mean'] - dbp_table.loc[subject_id, 'BP mean'])
    map_table.loc[subject_id, 'N'] = dbp_table.loc[subject_id, 'N']

# Compute averages and standard deviations for each table
for table in [dbp_table, sbp_table, map_table]:
    table.loc['Average'] = table.mean(numeric_only=True)
    table.loc['STD'] = table.std(numeric_only=True)



# Output the final tables with Subject ID as the index
print("Diastolic Blood Pressure (DBP) Table:")
print(dbp_table)
print("\nSystolic Blood Pressure (SBP) Table:")
print(sbp_table)
print("\nMean Arterial Pressure (MAP) Table:")
print(map_table)

Diastolic Blood Pressure (DBP) Table:
                  ME        Sde  95% CI +  95% CI -         r       RMSE  \
Subject ID                                                                 
1           0.368351   9.535341  0.791796 -0.055095 -0.411776   9.540007   
2          -0.007804   9.535024  0.315134 -0.330743 -0.459632   9.533604   
3          -1.658941   7.152975 -1.359697 -1.958185 -0.018519   7.341242   
4          -0.035760  12.182090  0.400972 -0.472492  0.472376  12.180105   
5           0.219817  14.200003  0.799651 -0.360016  0.165675  14.198622   
6          -1.326893   7.438930 -0.992221 -1.661564  0.601016   7.554414   
Average    -0.406872  10.007394 -0.007394 -0.806349  0.058190  10.057999   
STD         0.785791   2.499932  0.852391  0.725646  0.402628   2.444044   

             BP Range    BP mean            N  
Subject ID                                     
1           45.552252  83.493821  1948.000000  
2           44.972805  94.446071  3349.000000  
3        

In [ ]:
import pandas as pd
import os

# Define paths to the detailed and summary data
base_detailed_dir = "/Users/oe/BPS_Project/Graphene_BP/predictions_svr/SVM_Valsalva"
summary_data_path = "/Users/oe/BPS_Project/Graphene_BP/predictions_svr/valsalva.csv"

# Function to process detailed data for a single subject
def process_subject_data(subject_id, bp_type='DBP'):
    file_path = os.path.join(base_detailed_dir, f"subject_id_{subject_id}", "NoShuffleCV_valsalva_NoIBI_svr_mean_test.csv")
    detailed_data = pd.read_csv(file_path)
    errors = detailed_data[f'Predict_data_{bp_type}'] - detailed_data[f'Ref_data_{bp_type}']
    sde = errors.std()
    bp_mean = detailed_data[f'Ref_data_{bp_type}'].mean()
    bp_range = detailed_data[f'Ref_data_{bp_type}'].max() - detailed_data[f'Ref_data_{bp_type}'].min()
    return sde, bp_mean, bp_range

# Load the summary data
summary_data = pd.read_csv(summary_data_path).set_index('Subject ID')

# Initialize tables for DBP, SBP, and MAP
dbp_table = pd.DataFrame(index=summary_data.index)
sbp_table = pd.DataFrame(index=summary_data.index)
map_table = pd.DataFrame(index=summary_data.index)

# Process detailed data and compute summary statistics
for subject_id in summary_data.index:
    dbp_sde, dbp_mean, dbp_range = process_subject_data(subject_id, 'DBP')
    sbp_sde, sbp_mean, sbp_range = process_subject_data(subject_id, 'SBP')
    
    # Populate DBP table
    dbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.1']
    dbp_table.loc[subject_id, 'Sde'] = dbp_sde
    dbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.1'] + 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.1'] - 1.96 * dbp_sde / (summary_data.loc[subject_id, 'testing DBP N'] ** 0.5)
    dbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.1']
    dbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.1']
    dbp_table.loc[subject_id, 'BP Range'] = dbp_range
    dbp_table.loc[subject_id, 'BP mean'] = dbp_mean
    dbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing DBP N']
    
    # Populate SBP table
    sbp_table.loc[subject_id, 'ME'] = summary_data.loc[subject_id, 'ME.3']
    sbp_table.loc[subject_id, 'Sde'] = sbp_sde
    sbp_table.loc[subject_id, '95% CI +'] = summary_data.loc[subject_id, 'ME.3'] + 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, '95% CI -'] = summary_data.loc[subject_id, 'ME.3'] - 1.96 * sbp_sde / (summary_data.loc[subject_id, 'testing SBP N'] ** 0.5)
    sbp_table.loc[subject_id, 'r'] = summary_data.loc[subject_id, 'CC.3']
    sbp_table.loc[subject_id, 'RMSE'] = summary_data.loc[subject_id, 'RMSE.3']
    sbp_table.loc[subject_id, 'BP Range'] = sbp_range
    sbp_table.loc[subject_id, 'BP mean'] = sbp_mean
    sbp_table.loc[subject_id, 'N'] = summary_data.loc[subject_id, 'testing SBP N']


    
    # Calculate and populate MAP table by the standard formula (MAP = DBP + 1/3 (SBP - DBP)) from the DBP and SBP tables
    map_table.loc[subject_id, 'ME'] = dbp_table.loc[subject_id, 'ME'] + 1/3 * (sbp_table.loc[subject_id, 'ME'] - dbp_table.loc[subject_id, 'ME'])
    map_table.loc[subject_id, 'Sde'] = (dbp_table.loc[subject_id, 'Sde'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'Sde'] ** 2) ** 0.5
    map_table.loc[subject_id, '95% CI +'] = map_table.loc[subject_id, 'ME'] + 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, '95% CI -'] = map_table.loc[subject_id, 'ME'] - 1.96 * map_table.loc[subject_id, 'Sde'] / (dbp_table.loc[subject_id, 'N'] ** 0.5)
    map_table.loc[subject_id, 'r'] = (dbp_table.loc[subject_id, 'r'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'r'] ** 2) ** 0.5
    map_table.loc[subject_id, 'RMSE'] = (dbp_table.loc[subject_id, 'RMSE'] ** 2 + 1/9 * sbp_table.loc[subject_id, 'RMSE'] ** 2) ** 0.5
    map_table.loc[subject_id, 'BP Range'] = dbp_table.loc[subject_id, 'BP Range'] + 1/3 * (sbp_table.loc[subject_id, 'BP Range'] - dbp_table.loc[subject_id, 'BP Range'])
    map_table.loc[subject_id, 'BP mean'] = dbp_table.loc[subject_id, 'BP mean'] + 1/3 * (sbp_table.loc[subject_id, 'BP mean'] - dbp_table.loc[subject_id, 'BP mean'])
    map_table.loc[subject_id, 'N'] = dbp_table.loc[subject_id, 'N']

# Compute averages and standard deviations for each table
for table in [dbp_table, sbp_table, map_table]:
    table.loc['Average'] = table.mean(numeric_only=True)
    table.loc['STD'] = table.std(numeric_only=True)



# Output the final tables with Subject ID as the index
print("Diastolic Blood Pressure (DBP) Table:")
print(dbp_table)
print("\nSystolic Blood Pressure (SBP) Table:")
print(sbp_table)
print("\nMean Arterial Pressure (MAP) Table:")
print(map_table)